In [0]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.cuda import FloatTensor, LongTensor

import numpy as np

np.random.seed(42)

# Рекуррентные сети, часть 2

## POS Tagging

Мы уже посмотрели на примеры применения рекуррентных сетей.

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg =x250)

[(from The Unreasonable Effectiveness of Recurrent Neural Networks)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

---

Перейдем к ещё одному варианту - sequence labeling (последняя картинка).

Самые популярные примеры для такой постановки задачи - Part-of-Speech Tagging и Named Entity Recognition.

Мы порешаем сейчас POS-Tagging для английского.

Будем работать с таким набором тегов:
- ADJ - adjective (new, good, high, ...)
- ADP - adposition (on, of, at, ...)
- ADV - adverb (really, already, still, ...)
- CONJ - conjunction (and, or, but, ...)
- DET - determiner, article (the, a, some, ...)
- NOUN - noun (year, home, costs, ...)
- NUM - numeral (twenty-four, fourth, 1991, ...)
- PRT - particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- . - punctuation marks (. , ;)
- X - other (ersatz, esprit, dunno, ...)

Скачаем данные:

In [0]:
import nltk
import sys
import numpy as np
from sklearn.cross_validation import train_test_split

nltk.download('brown')
nltk.download('universal_tagset')

data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['ADV', 'NOUN', 'ADP', 'PRON', 'DET',  '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

Пример размеченного предложения:

In [0]:
for word, tag in data[0]:
    print('{:15}\t{}'.format(word, tag))

Напомню, на прошлом занятии мы строили LSTM сеть, которая обрабатывала последовательности символов, и предсказывала, к какому языку относится слово. 

LSTM выступал в роли feature extractor'а, работающего с произвольного размера последовательностью символов (ну, почти произвольного - мы ограничивались максимальной длиной слова). Батч для сети имел размерность `(max_word_len, batch_size)`.

Теперь мы опять хотим использовать такую же идею для извлечения признаков из последовательности символов - потому что последовательность символов же должна быть полезной для предсказания части речи, правда?

Сеть должна будет запомнить, например, что `-ly` - это часто про наречие, а `-tion` - про существительное.

Иллюстрация процесса:  
![Char BiLSTM](https://image.ibb.co/fYDk9c/Fig2.png =x400)

Но, конечно, только признаки, задаваемые набором символов в слове, совершенно бесполезны в многих случаях. Например, из-за омонимии:  
*“he cashed a check at the **bank**”*  
vs  
*“he sat on the **bank** of the river”*

Поэтому нам очень полезно учитывать контекст при предсказании тега.

В результате у нас есть сразу две последовательности: последовательность слов в предложении и последовательность символов в словах.  
*we_need_to_go_deeper.jpg*  

Будем использовать словный LSTM, который работает на результатах работы символьного LSTM.

Батч будет иметь размерности `(max_sentence_len, batch_size, max_word_len)`.

Всё вместе будет работать так:

![LSTM](https://www.researchgate.net/profile/Wang_Ling/publication/280912217/figure/fig1/AS:391505383575554@1470353565232/Illustration-of-our-neural-network-for-Language-Modeling.png =x450)  
from [Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation](https://arxiv.org/abs/1508.02096)

---

Оценим, для начала `max_word_len`: 

In [0]:
from collections import Counter 
    
def find_max_len(counter, threshold):
  sum_count = sum(counter.values())
  cum_count = 0
  for i in range(max(counter)):
    cum_count += counter[i]
    if cum_count > sum_count * threshold:
      return i
  return max(counter)

word_len_counter = Counter()
for sent in train_data:
  for word in sent:
    word_len_counter[len(word[0])] += 1
    
threshold = 0.99
MAX_WORD_LEN = find_max_len(word_len_counter, threshold)

print('Max word len for {:.0%} of words is {}'.format(threshold, MAX_WORD_LEN))

Сделаем не слишком хорошую вещь. Выкинем слишком длинные предложения

In [0]:
sent_len_counter = Counter()
for sent in train_data:
  sent_len_counter[len(sent)] += 1

threshold = 0.87
MAX_SENT_LEN = find_max_len(sent_len_counter, threshold)

print('Max sent length of {:.0%} of sentences is {}'.format(threshold, MAX_SENT_LEN))

In [0]:
train_data_len = len(train_data)
train_data = [sent for sent in train_data if len(sent) <= MAX_SENT_LEN]
print('Removed', train_data_len - len(train_data), 'too long sentences from train')

test_data_len = len(test_data)
test_data = [sent for sent in test_data if len(sent) <= MAX_SENT_LEN]
print('Removed', test_data_len - len(test_data), 'too long sentences from test')

In [0]:
print('Words count in train set:', sum(len(sent) for sent in train_data))
print('Words count in test set:', sum(len(sent) for sent in test_data))

Остаться должно вполне достаточно (но считаться всё будет быстрее).

Теперь нужно сконвертировать данные.

In [0]:
from string import punctuation

def get_range(first_symb, last_symb):
    return set(chr(c) for c in range(ord(first_symb), ord(last_symb) + 1))

chars = get_range('a', 'z') | get_range('A', 'Z') | get_range('0', '9') | set(punctuation)
char_index = {c : i for i, c in enumerate(chars)}

def get_char_index(char, char_index):
    return char_index[char] if char in char_index else len(char_index)

def convert_data(data, max_sent_len, max_word_len):
    X = np.zeros(<choose the dimensions>)
    y = np.zeros(<choose the dimensions>)
    for i, sent in enumerate(data):
        for j, (word, tag) in enumerate(sent):
            word = word[-max_word_len:]
            <add word into X, y>
    return X, y
  
X_train, y_train = convert_data(train_data, MAX_SENT_LEN, MAX_WORD_LEN)
X_test, y_test = convert_data(test_data, MAX_SENT_LEN, MAX_WORD_LEN)

Напишем генератор батчей (какую размерность они должны иметь?):

In [0]:
def iterate_batches(dataset, batch_size):
    """
    returns X_batch - Variable with size (max_sentence_len, batch_size, max_word_len)
    y_batch - Variable with size (max_sentence_len, batch_size)
    """
    X, y = dataset
    n_samples = X.shape[1]

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
        
        X_batch = ...
        y_batch = ...
        
        yield Variable(X_batch), Variable(y_batch)

Напишем достаточно удобный аналог метода `fit` в keras:

In [0]:
import math
import time

def do_epoch(model, criterion, data, batch_size, optimizer=None):
    epoch_loss = 0
    correct_count = 0
    sum_count = 0
    
    model.train(not optimizer is None)
    
    batchs_count = math.ceil(data[0].shape[1] / batch_size)
    
    for i, (X_batch, y_batch) in enumerate(iterate_batches(data, batch_size)):
        logits = model(X_batch)
        
        <calculate loss>
        epoch_loss += loss.data[0]
        
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        <calculate accuracy>
        
        correct_count += cur_correct_count
        sum_count += cur_sum_count
        
        print('\r[{} / {}]: Loss = {:.5f}, Accuracy = {:.2%}'.format(i, batchs_count, 
              loss.data[0], cur_correct_count / cur_sum_count), end='')

    return epoch_loss / batchs_count, correct_count / sum_count

def fit(model, criterion, optimizer, train_data, epochs_count=1, 
        batch_size=32, val_data=None, val_batch_size=None):
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        start_time = time.time()
        train_loss, train_acc = do_epoch(model, criterion, train_data, batch_size, optimizer)
        
        output_info = '\rEpoch {} / {}, Epoch Time = {:.2f}s: Train Loss = {:.5f}, Train Accuracy = {:.2%}'
        if not val_data is None:
            val_loss, val_acc = do_epoch(model, criterion, train_data, batch_size, None)
            epoch_time = time.time() - start_time
            output_info += ', Val Loss = {:.5f}, Val Accuracy = {:.2%}'
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss, train_acc, val_loss, val_acc))
        else:
            epoch_time = time.time() - start_time
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss, train_acc))

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, char_hidden_dim, 
                 lstm_hidden_dim, lstm_layers_count, tagset_size):
        super().__init__()
        
        <init layers>

    def forward(self, chars):
        <calculate logits>

In [0]:
CHAR_VOCAB_SIZE = len(char_index) + 1
CHAR_EMB_DIM = 24
CHAR_HIDDEN_DIM = 64
LSTM_HIDDEN_DIM = 128
LSTM_LAYER_COUNT = 1

model = LSTMTagger(<params>)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=20, 
    batch_size=128, val_data=(X_test, y_test), val_batch_size=2048)

### Masking
Что, собственно, мы оптимизируем? Мы считаем сумму потерь для `max_sentence_len x batch_size` слов. Но ведь многие из них - нули.

В keras есть специальный слой, который помогает не считать потери для нулевых сэмплов - [`Masking`](https://keras.io/layers/core/#masking).

Хочется примерно такого же добиться. Для этого достаточно просто находить маску - матрицу из нулей и единиц с нулями там, где стоят нулевые элементы `y_batch`.  
Дальше эту маску нужно умножить на потери для каждого элемента - и усреднить. В итоге потери для паддингов занулятся и не будут участвовать в подсчете потерь.

**Задание** Добавьте Masking.

### Bidirectional LSTM

Благодаря BiLSTM можно использовать сразу оба контеста при предсказании тега слова. Т.е. для каждого токена $w_i$ forward LSTM будет выдавать представление $\mathbf{f_i} \sim (w_1, \ldots, w_i)$ - построенное по всему левому контексту - и $\mathbf{b_i} \sim (w_n, \ldots, w_i)$ - представление правого контекста. Их конкатенация автоматически захватит весь доступный контекст слова: $\mathbf{h_i} = [\mathbf{f_i}, \mathbf{b_i}] \sim (w_1, \ldots, w_n)$.

![BiLSTM](https://www.researchgate.net/profile/Wang_Ling/publication/280912217/figure/fig2/AS:391505383575555@1470353565299/Illustration-of-our-neural-network-for-POS-tagging.png =x450)  
from [Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation](https://arxiv.org/abs/1508.02096)

**Задание** Добавьте Bidirectional LSTM.

### Словные эмбеддинги

**Задание** Только символьных эмбеддингов может быть недостаточно. Добавьте ещё словные эмбеддинги. Слова стоит приводить к нижнему регистру - признаки, связанные с регистром должны ухватываться символьный LSTM.

Эти эмбеддинги можно просто сконкатенировать, а можно использовать гейт (как в LSTM). Например, по эмбеддингу слова предсказывать $o = \sigma(w)$ - насколько он хорош и сочетать в такой пропорции с символьным эмбеддингом: $o \odot w + (1 - o) \odot \tilde w$, где $\tilde w$ - эмбеддинг слова, полученный по символьному уровню.

### Byte-Pair Encoding

Мы можем представлять слова одним индексом - и использовать словные эмбеддинги как строки матрицы эмбеддингов.  
Можем считать их набором символов и получать словный эмбеддинг с помощью некоторой функции над символьными эмбеддингами.

Наконец, можем ещё использовать промежуточное представление - как набор подслов.

Несколько лет назад использование подслов предложили для задачи машинного перевода: [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909). Там использовалось byte-pair encoding.

По сути это процесс объединения самых частотных пар символов алфавита в новый суперсимвол. Пусть у нас есть словарь, состоящий из такого набора слов:  
`‘low·’, ‘lowest·’, ‘newer·’, ‘wider·’`   
(`·` означает конец слова)

Тогда первым может выучиться новый символ `r·`, за ним `l o` превратится в `lo`. К этому новому символу приклеится `w`: `lo w` $\to$ `low`. И так далее.

Утверждается, что таким образом выучатся, во-первых, все частотные и короткие слова, а во-вторых, все значимые подслова. Например, в полученном в результате алфавите должны найтись `ly·` и `tion·`.

Дальше слово можно разбить на набор подслов - и действовать, как с символами.

Здесь можно найти уже предобученные эмбеддинги: [BPEmb](https://github.com/bheinzerling/bpemb).

**Задание* ** Посмотреть на то, какие подслова выделились для английского. Попробовать добавить такие эмбеддинги в модель (или хотя бы описать способ, как это можно сделать).

### Полносвязная сеть

Вообще говоря, интересно поставить такой бейзлайн: какое качество получит полносвязная сеть, которая будет просто использовать 2-3 слова слева и 2-3 слова справа для предсказания. Для слов в начале предложения можно использовать те же паддинги.

В результате батч, с которым мы имеет дело, будет иметь размерность `(batch_size, 5, max_word_len)`, а предсказывать мы будем вектор размером `(batch_size,)`. При этом `batch_size` здесь будет уже заметно больше.

**Задание** Реализуйте полносвязную сеть.

Вообще говоря, сейчас в принципе пытаются отказываться от рекуррентных сетей в пользу более параллелизуемым. Например, для того же pos-tagging'а в прошлом году была такая статья: [Natural Language Processing with Small Feed-Forward Networks](https://arxiv.org/abs/1708.00214)

## Dropout

Вспомним, что такое dropout.

По сути это умножение случайно сгенерированной маски из нулей и единиц на входной вектор (+ нормировка).

Например, для слоя Dropout(p):

$$m = \frac1{1-p} \cdot \text{Bernouli}(1 - p)$$
$$\tilde h = m \odot h $$

В рекуррентных сетях долго не могли прикрутить dropout. Делать это пытались, генерируя случайную маску:   
![A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](https://cdn-images-1.medium.com/max/800/1*g4Q37g7mlizEty7J1b64uw.png =x300)  
from [A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](https://arxiv.org/abs/1512.05287)

Оказалось, правильнее делать маску фиксированную: для каждого шага должны зануляться одни и те же элементы.

Для pytorch нет нормального встроенного variational dropout в LSTM. Зато есть [AWD-LSTM](https://github.com/salesforce/awd-lstm-lm).

Советую посмотреть обзор разных способов применения dropout'а в рекуррентных сетях: [Dropout in Recurrent Networks — Part 1](https://becominghuman.ai/learning-note-dropout-in-recurrent-networks-part-1-57a9c19a2307) (в конце - ссылки на Part 2 и 3).

## Word-level Language Model

А теперь перейдем к языковому моделированию на словном уровне.

In [0]:
!wget --quiet -c train.txt https://raw.githubusercontent.com/yoonkim/lstm-char-cnn/master/data/ptb/train.txt
!wget --quiet -c valid.txt https://raw.githubusercontent.com/yoonkim/lstm-char-cnn/master/data/ptb/valid.txt
!wget --quiet -c test.txt https://raw.githubusercontent.com/yoonkim/lstm-char-cnn/master/data/ptb/test.txt

Будем работать с такими файлами:

In [0]:
!head train.txt

На каждой строке записано предложение. Все слова приведены к нижнему регистру, низкочастные заменены на <unk>, а числа - на N.

Преобразуем слова в них к набору индексов.

In [0]:
import os

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self, ):
        self.dictionary = Dictionary()
        self.train = self.tokenize('train.txt')
        self.valid = self.tokenize('valid.txt')
        self.test = self.tokenize('test.txt')

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids
      
corpus = Corpus()

**Задание** Начнём с бейзлайна: постройте простую N-граммную модель. 

Это можно сделать двумя способами: во-первых, с использованием стандартного `Counter` запомнить все встречающиеся в тексте N-граммы. При этом стоит добавлять паддинги - символы пустых слов в начале и конце предложений.

Второй вариант - полносвязная нейронная сеть, которая принимает индексы N слов, и предсказывает по ним (N+1)-ое. Тут вам сразу должен вспомниться CBOW с позапрошлого занятия. Собственно, словные эмбеддинги и начались с языкового моделирования. Вот [тут](https://ronan.collobert.com/senna/) можно посмотреть, какой была жизнь до word2vec ("Our embeddings have been trained for about 2 months, over Wikipedia").

Реализуйте оба варианта.

**Задание** Постройте модель, аналогичную той, что предсказывала символы.

Теперь модель должна быть уже заметно большего размера, чтобы реально выучить язык. Используйте хотя бы два слоя LSTM. Число слоев задается параметром `LSTM`.

Также стоит использовать `dropout` в LSTM.

Эмбеддинги можно обучать с нуля, а можно - использовать уже готовые. Будет неплохо, если вы попробуете оба варианта. Хотя, надо отметить, в языковом моделировании обычно используют обучаемые с нуля эмбеддинги.

Даже если вы будете учить эмбеддинги с нуля, стоит обратить внимание на их инициализацию. Кажется, лучше зайдет не вариант из pytorch по умолчанию, а инициализация из равномерного распределения из диапазона `(-0.05, 0.05)`.

Модель, в которой сразу есть сразу и входной эмбеддинг размера, скажем `200 x 10000` и выходной слой с такой же размерностью, получается очень уж тяжелой. Обратите внимание, что можно в качестве выходного слоя использовать тот же самый слой эмбеддингов (точнее, веса из него): [Using the Output Embedding to Improve Language Models](https://arxiv.org/abs/1608.05859).

В результате, эмбеддинги лучше обучаются потому, что выходной слой вообще учится гораздо лучше входного, а также потому, что этот выходной слой обновляется на каждом мини-батче, тогда как в слое эмбеддингов обновляются лишь строки, соответствующие словам в этом мини-батче.

Сравните N-граммные и рекуррентную языковые модели. Обратите внимание как на размер, так и на качество.

<бесстыдная самореклама> Подробнее почитать, в том числе и про то, как меряют качество (перплексию), можно здесь: [Как научить свою нейросеть генерировать стихи](https://habrahabr.ru/post/334046/) </бесстыдная самореклама>.